In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import re
import threading
import queue
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer
import sys

In [2]:
# Initialize vtk
reader = vtk.vtkSTLReader() # read stl file
reader.SetFileName("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\tool_stl_models\\medtronic_chicken_foot.STL") # set the file name
reader.Update() # update the reader

mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
mapper.SetInputData(reader.GetOutput()) # set the input to the reader

actor = vtk.vtkActor() # create an actor, an actor is used to display the data
actor.SetMapper(mapper) # set the mapper to the actor

renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(1000, 800) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.AddActor(actor) # add the actor to the renderer
renderer.SetBackground(0.1, 0.2, 0.4) # set the background of the renderer

# 创建坐标轴
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# 设置坐标轴标签的位置
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# 设置坐标轴标签
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# 将坐标轴添加到 renderer
renderer.AddActor(axes)

# 设定摄像机视角，固定在 (0, 0, -3000) 位置
renderer.GetActiveCamera().SetPosition(-1000, 1000, -5000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, 0)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围


In [3]:
# Polaris Data Streaming Thread
polaris_matrix = np.eye(4)
data_ready = threading.Event()
frame_idx = 0
# 线程安全的队列
data_queue = queue.Queue()

def read_polaris_data(recording, frame_idx):
    row = recording.iloc[frame_idx]
    tracking = row[3]  # 提取 tracking 数据
    numbers = [float(x) if x != "nan" else np.nan for x in re.findall(r"nan|[-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|\d+", tracking)]
    tracking_matrix = np.array(numbers).reshape(4, 4)
    # print(frame_idx)

    return tracking_matrix

def polaris_thread():
    global polaris_matrix
    global frame_idx
    while True:
        new_matrix = read_polaris_data(recording, frame_idx)
        data_queue.put(new_matrix)
        # polaris_matrix = new_matrix
        data_ready.set()
        frame_idx += 1
        time.sleep(1/400)
        print(frame_idx)


In [ ]:
# VTK Update Thread
def update_pose():
    global polaris_matrix

    if not data_queue.empty():
        polaris_matrix = data_queue.get()  # 取最新数据
    
        transform = vtk.vtkTransform()
        transform.SetMatrix(polaris_matrix.flatten()) 
        actor.SetUserTransform(transform)

        render_window.Render()
        # render_window_interactor.InvokeEvent("RenderEvent")            
        # data_ready.clear()

In [ ]:
recording = pd.read_csv('recording.csv', header=None)
tracking_thread = threading.Thread(target=polaris_thread, daemon=True)
tracking_thread.start()

# render_window_interactor.AddObserver("TimerEvent", update_pose)
# render_window_interactor.CreateRepeatingTimer(4)  # VTK 线程不断监听
app = QApplication(sys.argv)
timer = QTimer()
timer.timeout.connect(update_pose)
timer.start(1)  # 16ms ≈ 60FPS


render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.Start()

sys.exit(app.exec_())
